##Hello World Agent

In [ ]:
# Install the langgraph library
!pip install langgraph
# Import the required classes
from typing import Dict, TypedDict
from langgraph.graph import StateGraph

In [ ]:


# Define the state structure
class AgentState(TypedDict):
    message: str

# Define the greeting node as a separate function
def greeting_node(state: AgentState) -> AgentState:
    """simple node that adds a greeting message to the state"""
    state["message"] = "Hey " + state["message"] + ", how is your day going?"
    return state

In [ ]:
graph = StateGraph(AgentState)

graph.add_node("greeter", greeting_node)

graph.set_entry_point("greeter")

graph.set_finish_point("greeter")

app = graph.compile()

In [ ]:
from IPython.display import Image, display

# Call draw_mermaid_png() on the compiled graph 'app' to get the image data
graph_image_data = app.get_graph().draw_mermaid_png()

# Pass the image data to the Image constructor and then display it
display(Image(graph_image_data))

In [ ]:
result = app.invoke({"message": "Enock Abungana"})

In [ ]:
result['message']

## Graph II

In [ ]:
# Import the required classes
from typing import Dict, TypedDict, List
from langgraph.graph import StateGraph

In [ ]:
from os import name
class AgentState(TypedDict):
    values: List[int]
    name: str
    result: str

In [ ]:
def process_values(state: AgentState) -> AgentState:
    """simple node that adds a greeting message to the state"""
    state["result"] = str(sum(state["values"]))
    return state

In [ ]:
graph = StateGraph(AgentState)

graph.add_node("processor", process_values)

graph.set_entry_point("processor")

graph.set_finish_point("processor")

app = graph.compile()

In [ ]:
from IPython.display import Image, display

# Call draw_mermaid_png() on the compiled graph 'app' to get the image data
graph_image_data = app.get_graph().draw_mermaid_png()

In [ ]:
!pip install --upgrade langchain langchain-community langgraph openai faiss-cpu


In [ ]:
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
# 1. Install dependencies (once; run in your notebook or terminal):
#    %pip install -qU langgraph langchain langchain-community langchain-openai openai faiss-cpu typing-extensions

import os
from typing_extensions import TypedDict, Annotated

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langchain_openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS

# 2. Helper to chunk text
def split_text(text: str, chunk_size: int = 1500) -> list[str]:
    return [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]

# 3. Define shared state schema
class State(TypedDict):
    messages: Annotated[list, add_messages]
    resume_chunks: list[str]
    jd_text: str
    top_sections: list[str]
    rewritten_bullets: str
    cover_letter: str

# 4. Node implementations

def load_documents(state: State) -> State:
    # Dummy text so we don’t hit file errors
    dummy_resume = """
    • Designed and deployed a CNN model for flood prediction in Sudan, achieving 85% accuracy.
    • Built an Airflow ETL pipeline to ingest and process GIS settlement data daily.
    • Collaborated with UNHCR to develop a displacement-forecasting dashboard serving 3 million+ individuals.
    """
    dummy_jd = """
    We are seeking a Data Scientist to build LLM-powered workflows that integrate vector search,
    FAISS, and Python. Experience with iterative agent-based design and multi-step pipelines is a plus.
    """
    state["resume_chunks"] = split_text(dummy_resume)
    state["jd_text"] = dummy_jd.strip()
    return state

def retrieve_relevant_sections(state: State) -> State:
    api_key = os.getenv("OPENAI_API_KEY")
    emb = OpenAIEmbeddings(openai_api_key=api_key)
    vect = FAISS.from_documents(state["resume_chunks"], emb)
    docs = vect.similarity_search(state["jd_text"], k=5)
    state["top_sections"] = [doc.page_content for doc in docs]
    return state

def rewrite_for_target(state: State) -> State:
    api_key = os.getenv("OPENAI_API_KEY")
    llm = OpenAI(temperature=0.2, openai_api_key=api_key)
    prompt = (
        "Rewrite these résumé bullets to match the tone, keywords, and style of the job description below.\n\n"
        f"Job Description:\n{state['jd_text']}\n\n"
        "Résumé Sections:\n" + "\n".join(state["top_sections"])
    )
    state["rewritten_bullets"] = llm(prompt)
    return state

def assemble_output(state: State) -> State:
    state["cover_letter"] = (
        "Dear Hiring Manager,\n\n"
        "I am excited to apply for this position because…\n\n"
        f"{state['rewritten_bullets']}\n\n"
        "Thank you for your consideration.\n\n"
        "Sincerely,\nYour Name"
    )
    return state

# 5. Build and compile the graph
builder = StateGraph(State)
builder.add_node(load_documents)
builder.add_node(retrieve_relevant_sections)
builder.add_node(rewrite_for_target)
builder.add_node(assemble_output)

builder.set_entry_point("load_documents")
builder.add_edge(START,                    "load_documents")
builder.add_edge("load_documents",        "retrieve_relevant_sections")
builder.add_edge("retrieve_relevant_sections", "rewrite_for_target")
builder.add_edge("rewrite_for_target",    "assemble_output")
builder.add_edge("assemble_output",       END)

graph = builder.compile()

# 6. Execute
if __name__ == "__main__":
    # Ensure your OPENAI_API_KEY is set, e.g.:
    #   export OPENAI_API_KEY="sk-…"
    initial_state: State = {
        "messages": [],
        "resume_chunks": [],
        "jd_text": "",
        "top_sections": [],
        "rewritten_bullets": "",
        "cover_letter": "",
    }
    result = graph.invoke(initial_state)
    print("\n--- Generated Cover Letter ---\n")
    print(result["cover_letter"])


In [ ]:
##bash API Key set up
export OPENAI_API_KEY="your_openai_key_here"

